In [22]:
from db_utils import retrieve_data
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
path = r"c:\Users\brtnl\OneDrive\Desktop\code\sports_betting"
os.chdir(path)

In [ ]:
# basic ideas, maybe some month? month * indicator? 

In [8]:
query = """SELECT * FROM cfb.all_games"""
game_df = retrieve_data(query)

In [11]:
game_df["home_half"] = game_df["home_first_quarter"] + game_df["home_second_quarter"]
game_df["visitor_half"] = game_df["visitor_first_quarter"] + game_df["visitor_second_quarter"]

In [19]:
target_col = "home_second_quarter"
X = game_df.loc[:, game_df.columns != target_col]
y = game_df[target_col]

In [ ]:
# Indicators for the home and visitor team  
game_df = pd.concat([game_df, pd.get_dummies(game_df["home"], prefix="home"), pd.get_dummies(game_df["visitor"], prefix="vis")], axis=1)

In [35]:
categorical_cols = ["date", "home", "visitor", "ot", "unique_id", "created_at"]
game_df.loc[:, ~game_df.columns.isin(categorical_cols)].fillna(0)

,home_points,visitor_points,home_first_quarter,home_second_quarter,home_third_quarter,home_fourth_quarter,home_ot_total,visitor_first_quarter,visitor_second_quarter,visitor_third_quarter,visitor_fourth_quarter,visitor_ot_total,home_rank,visitor_rank,home_half,visitor_half,home_Air Force,home_Akron,home_Alabama,home_Appalachian State,home_Arizona,home_Arizona State,home_Arkansas,home_Arkansas State,home_Army,home_Auburn,home_Ball State,home_Baylor,home_Boise State,home_Boston College,home_Bowling Green State,home_Brigham Young,home_Buffalo,home_California,home_Campbell,home_Central Michigan,home_Charleston Southern,home_Charlotte,home_Cincinnati,home_Clemson,...,vis_Texas Tech,vis_The Citadel,vis_Toledo,vis_Towson,vis_Troy,vis_Tulane,vis_Tulsa,vis_UAB,vis_UCF,vis_UCLA,vis_UNLV,vis_USC,vis_UTEP,vis_UTSA,vis_Utah,vis_Utah State,vis_Utah Tech,vis_Valparaiso,vis_Vanderbilt,vis_Villanova,vis_Virginia,vis_Virginia Military Institute,vis_Virginia Tech,vis_Wagner,vis_Wake Forest,vis_Washington,vis_Washington State,vis_Weber State,vis_Wesley,vis_West Virginia,vis_Western Carolina,vis_Western Illinois,vis_Western Kentucky,vis_Western Michigan,vis_William & Mary,vis_Wisconsin,vis_Wofford,vis_Wyoming,vis_Yale,vis_Youngstown State
0,54,52,7,17,0,22,8,10,7,21,8,6,0.0,0.0,24,17,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,30,26,14,0,10,6,0,3,14,6,3,0,0.0,0.0,14,17,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,17,34,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,53,7,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,40,35,0,0,0,0,0,0,0,0,0,0,0.0,8.0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,28,29,0,0,0,0,0,0,0,0,0,0,4.0,0.0,0,0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F